In [ ]:
import json
import os
import datetime as dt
from dateutil.relativedelta import relativedelta
from glob import glob
from pprint import pprint

import fbchat
import pandas as pd

In [ ]:
creds = json.load(open("./local/creds.json"))
client = fbchat.Client(creds["id"], creds["password"])

In [ ]:
def ts_to_dt(ts):
    ts = int(ts)
    # assume ms if > 2025
    if ts > 1735718400:
        ts /= 1000
    return dt.datetime.fromtimestamp(ts).date()

In [ ]:
def get_df_api(friend):
    """Get message data from the FB messenger API (which seems buggy)
    """
    # messages after this date won't be included
    before_ts = dt.datetime.timestamp(dt.datetime(2019, 12, 20)) * 1000

    friend_obj, = client.searchForUsers(friend, 1)
    assert friend_obj.is_friend
    thread_info = client.fetchThreadInfo(friend_obj.uid)
    num_messages = thread_info[friend_obj.uid].message_count
    print(num_messages, " messages")
    messages = client.fetchThreadMessages(friend_obj.uid, limit=num_messages)

    min_month = ts_to_dt(messages[-1].timestamp).replace(day=1)
    max_month = ts_to_dt(messages[0].timestamp).replace(day=1)

    date = min_month
    end = max_month
    dates = []
    while date <= max_month:
        dates.append(date)
        date += relativedelta(months=1)

    counts = []
    for date in dates:
        count = len([x for x in messages if date == ts_to_dt(x.timestamp).replace(day=1)])
        counts.append(count)
        date += relativedelta(months=1)

    df = pd.DataFrame(counts, index=dates)
    return df

In [ ]:
data_dir = os.path.join(os.path.expanduser(<your fb data location>), "messages/inbox")

def get_df_files(friend):
    """Get message data from downloaded FB data dump
    """
    for x in os.listdir(data_dir):
        # not 100% correct: group chats can start with someone's name
        if x.lower().startswith(friend.replace(" ", "").lower()):
            dir_ = x
            break
    print(dir_)

    files = glob(os.path.join(data_dir, dir_, "message_*.json"))

    messages = []
    for file in sorted(files):
        blob = json.load(open(file))
        messages += blob["messages"]

    min_month = ts_to_dt(messages[-1]["timestamp_ms"]).replace(day=1)
    max_month = ts_to_dt(messages[0]["timestamp_ms"]).replace(day=1)

    date = min_month
    end = max_month
    dates = []
    while date <= max_month:
        dates.append(date)
        date += relativedelta(months=1)

    counts = []
    for date in dates:
        count = len([x for x in messages if date == ts_to_dt(x["timestamp_ms"]).replace(day=1)])
        counts.append(count)
        date += relativedelta(months=1)

    df = pd.DataFrame(counts, index=dates)
    return df

In [ ]:
get_df_files(<friend name here>).plot()